In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [7]:
# defining endpoint and parameters
url = 'https://www.sec.gov/cgi-bin/browse-edgar'
params = {'action': 'getcompany', 
          'CIK': 66740, 
          'type': '8-K', 
          'output':'xml', 
          'dateb': '20200430',
          'datea': '20150501',
          'start': '',
          'count': '100'}

# response
sec_response = requests.get(url=url, params=params)

# printing status code and url to check if its working
print(sec_response.status_code)
print(sec_response.url)

# creating soup
soup = BeautifulSoup(sec_response.content, 'xml')

200
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=66740&type=8-K&output=xml&datea=20150501&start=&count=100
